<a href="https://colab.research.google.com/github/ZhechengLiao/ML-algorithm/blob/master/LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build Logistic from Scratch

In [242]:
import numpy as np
from numpy.linalg import inv
from sklearn import datasets

In [243]:
iris = datasets.load_iris()
x_train = iris.data[:-50, :2]  # we only take the first two features.
y_train = iris.target[:-50]

In [252]:
def sigmoid(w, x):
  z = w.dot(x.T)
  return 1.0 / (1.0 + np.exp(-z))

def loss(y, y_pred):
  return -np.mean(y*np.log(y_pred) + (1-y)*np.log(1-y_pred))

def gradient(x, y, y_pred):
  return 1/x.shape[0] * np.dot(x.T, (y_pred-y))

def hessian(x):
  return 1/x.shape[0] * np.sum(np.dot(sigmoid(w, x)*(1-sigmoid(w, x)), x.dot(x.T)))

def newton(x, y, y_pred):
  # f'(x) / f''(x) => grad / H
  return gradient(x, y, y_pred) / hessian(x)

In [255]:
w = np.zeros(2)
lr = 0.1
epoch_num = 10
loss(y_train, sigmoid(w, x_train))

0.6931471805599453

In [258]:
for epoch in range(epoch_num):
  grad = gradient(x_train, y_train, sigmoid(w, x_train))
  w -= lr*grad
  l = loss(y_train, sigmoid(w, x_train))
  print(f'epoch: {epoch + 1}, loss: {l}, weight: {w}')

epoch: 1, loss: 0.6248255301804133, weight: [ 0.14528919 -0.23305451]
epoch: 2, loss: 0.6193129814982582, weight: [ 0.15683884 -0.25355026]
epoch: 3, loss: 0.6138882652113291, weight: [ 0.16829707 -0.27388142]
epoch: 4, loss: 0.608549815425159, weight: [ 0.17966478 -0.29404964]
epoch: 5, loss: 0.6032960859969277, weight: [ 0.19094285 -0.31405656]
epoch: 6, loss: 0.5981255509739806, weight: [ 0.20213218 -0.3339038 ]
epoch: 7, loss: 0.5930367049802403, weight: [ 0.21323366 -0.35359301]
epoch: 8, loss: 0.5880280635527795, weight: [ 0.22424817 -0.37312582]
epoch: 9, loss: 0.5830981634308278, weight: [ 0.23517662 -0.39250388]
epoch: 10, loss: 0.5782455627994861, weight: [ 0.24601989 -0.4117288 ]


# Newton Method

In [259]:
for epoch in range(epoch_num):
  w -= newton(x_train, y_train, sigmoid(w, x_train))
  l = loss(y_train, sigmoid(w, x_train))
  print(f'epoch:{epoch}, loss: {l}, weight: {w}')

epoch:0, loss: 0.5781960357602896, weight: [ 0.24613101 -0.4119258 ]
epoch:1, loss: 0.578146515870384, weight: [ 0.24624212 -0.41212278]
epoch:2, loss: 0.5780970031283117, weight: [ 0.24635323 -0.41231975]
epoch:3, loss: 0.578047497532615, weight: [ 0.24646433 -0.41251671]
epoch:4, loss: 0.577997999081837, weight: [ 0.24657542 -0.41271365]
epoch:5, loss: 0.577948507774521, weight: [ 0.24668651 -0.41291058]
epoch:6, loss: 0.577899023609211, weight: [ 0.24679759 -0.4131075 ]
epoch:7, loss: 0.5778495465844504, weight: [ 0.24690866 -0.41330441]
epoch:8, loss: 0.5778000766987845, weight: [ 0.24701973 -0.4135013 ]
epoch:9, loss: 0.5777506139507574, weight: [ 0.24713078 -0.41369819]


# Pytorch logistic regression

In [279]:
from torch import nn
import torch
iris = datasets.load_iris()
x_train = torch.tensor(iris.data[:-50, :2], dtype=torch.float32)  # we only take the first two features.
y_train = torch.tensor(iris.target[:-50], dtype=torch.float32)

In [280]:
class LogisticRegression(torch.nn.Module):
  def __init__(self, inputDim, outputDim):
    super(LogisticRegression, self).__init__()
    self.linear = torch.nn.Linear(inputDim, outputDim)

  def forward(self, x):
    output = torch.sigmoid(self.linear(x))
    return output

In [281]:
model = LogisticRegression(2, 1)
lr = 0.1
loss = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
epoch_num = 10

In [282]:
for epoch in range(epoch_num):
  optimizer.zero_grad()
  output = model(x_train)
  l = loss(torch.squeeze(output), y_train)
  l.backward()
  optimizer.step()
  print(f'epoch:{epoch+1}, loss:{l}, weigts:{[(w, b) for w, b in model.named_parameters()]}')

epoch:1, loss:1.835126519203186, weigts:[('linear.weight', Parameter containing:
tensor([[ 0.4575, -0.0856]], requires_grad=True)), ('linear.bias', Parameter containing:
tensor([-0.1800], requires_grad=True))]
epoch:2, loss:1.0330897569656372, weigts:[('linear.weight', Parameter containing:
tensor([[ 0.2692, -0.2204]], requires_grad=True)), ('linear.bias', Parameter containing:
tensor([-0.2183], requires_grad=True))]
epoch:3, loss:0.6400909423828125, weigts:[('linear.weight', Parameter containing:
tensor([[ 0.2144, -0.2779]], requires_grad=True)), ('linear.bias', Parameter containing:
tensor([-0.2321], requires_grad=True))]
epoch:4, loss:0.6038058400154114, weigts:[('linear.weight', Parameter containing:
tensor([[ 0.2243, -0.2986]], requires_grad=True)), ('linear.bias', Parameter containing:
tensor([-0.2341], requires_grad=True))]
epoch:5, loss:0.5985633134841919, weigts:[('linear.weight', Parameter containing:
tensor([[ 0.2357, -0.3182]], requires_grad=True)), ('linear.bias', Paramete

In [275]:
for weight, bias in model.named_parameters():
  print(weight, bias)

linear.weight Parameter containing:
tensor([[-0.3982,  0.1895]], requires_grad=True)
linear.bias Parameter containing:
tensor([-0.2033], requires_grad=True)
